**Connect google drive**

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Feb 22 08:55:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 20220211_165529.jpg	      'Colab Notebooks'		       yolov3
 B1_Sagnnik_Biswas_200907098  'Document from Sagnnik Biswas'   yolov4
 Chem.zip		      'My Drive'


**1) Clone the Darknet**



In [5]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15395, done.
remote: Total 15395 (delta 0), reused 0 (delta 0), pack-reused 15395
Receiving objects: 100% (15395/15395), 14.02 MiB | 2.30 MiB/s, done.
Resolving deltas: 100% (10352/10352), done.


**2) Compile Darknet using Nvidia GPU**


In [6]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [10]:
!echo "BOH" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-02-22 08:57:35--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.9MB/s    in 8.3s    

2022-02-22 08:57:44 (18.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
!unzip /mydrive/yolov3/BOH.zip -d data/obj

Archive:  /mydrive/yolov3/BOH.zip
  inflating: data/obj/102_png.rf.20ca210105b9d4d4b2ad490abf719b26.txt  
  inflating: data/obj/102_png.rf.22af1a2e170c85e6072a7a044d651c3e.jpg  
  inflating: data/obj/102_png.rf.22af1a2e170c85e6072a7a044d651c3e.txt  
  inflating: data/obj/102_png.rf.801b0124bf0ad1713fe7ed587dd5dd1e.jpg  
  inflating: data/obj/102_png.rf.801b0124bf0ad1713fe7ed587dd5dd1e.txt  
  inflating: data/obj/103_png.rf.116ec3eb3e2b147b3257af66b6f001e4.jpg  
  inflating: data/obj/103_png.rf.116ec3eb3e2b147b3257af66b6f001e4.txt  
  inflating: data/obj/103_png.rf.404291514d554621a31ff6c0bfeb08db.jpg  
  inflating: data/obj/103_png.rf.404291514d554621a31ff6c0bfeb08db.txt  
  inflating: data/obj/103_png.rf.d13b69f1c64d651f77400e507dbe8b06.jpg  
  inflating: data/obj/103_png.rf.d13b69f1c64d651f77400e507dbe8b06.txt  
 extracting: data/obj/classes.txt    
  inflating: data/obj/0_png.rf.5466ac215ece8ca5510458a945a93525.jpg  
  inflating: data/obj/0_png.rf.5466ac215ece8ca5510458a945a93525.tx

In [13]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/11_png.rf.6519fbf75e9da250a2f60edeb1c7b533.jpg', 'data/obj/8_png.rf.2131e8d96dba761e2f0b6ceed0acb30e.jpg', 'data/obj/78_png.rf.49d1fd3857c6133d022e1015ca177bc7.jpg', 'data/obj/2_png.rf.2fff19facdc20e01ae47979cc831713e.jpg', 'data/obj/34_png.rf.cfab1e49f24b60876748e1cf920e7546.jpg', 'data/obj/6_png.rf.f5b1658d84fc08bc08baab12d61a9273.jpg', 'data/obj/32_png.rf.cb100c41cea723e913240a711084d9db.jpg', 'data/obj/17_png.rf.4370daaf197d54030d555a6781dcf0f8.jpg', 'data/obj/12_png.rf.1ac7606cc0a7f512a3e1adb401112424.jpg', 'data/obj/14_png.rf.59add032f3567274307000f8b12cf32d.jpg', 'data/obj/26_png.rf.971a9e29d23262194a7436e3ac7a8906.jpg', 'data/obj/87_png.rf.19a0e7b5d3bc0fdb4e46891fd30d7a40.jpg', 'data/obj/91_png.rf.0d89c46797c67480645c8bea305eb24d.jpg', 'data/obj/46_png.rf.6c6d701a5a7b4995744364a4496f96b1.jpg', 'data/obj/43_png.rf.64dbaf1615103fff5bbdb5916beb33e1.jpg', 'data/obj/36_png.rf.84a9a1122980977b2582392d407166ad.jpg', 'data/obj/11_png.rf.c131a3d9eaa1d17bcb1b6160343ad31d.jpg',

In [14]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.530327), count: 4, class_loss = 0.274693, iou_loss = 0.351103, total_loss = 0.625796 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
 total_bbox = 60595, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.481812), count: 4, class_loss = 0.634322, iou_loss = 0.701618, total_loss = 1.335940 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000001, iou_loss = 0.000000, total_loss = 0.000001 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00